<a href="https://colab.research.google.com/github/BenShieh233/NLP-Text-Mining/blob/main/IC8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import string
import nltk
import numpy as np # linear algebra
nltk.download('vader_lexicon')
from nltk.util import ngrams
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud,STOPWORDS
import spacy as sp
nlps = sp.load("en_core_web_sm")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
tesla_tw = pd.read_csv('/content/drive/MyDrive/INFOSYS765/Tesla_stock_price_2016.csv')
tesla_tw.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,1/4/2016,230.720001,231.380005,219.000000,223.410004,6827100,223.410004
1,1/5/2016,226.360001,226.889999,220.000000,223.429993,3186800,223.429993
2,1/6/2016,220.000000,220.050003,215.979996,219.039993,3779100,219.039993
3,1/7/2016,214.190002,218.440002,213.669998,215.649994,3554300,215.649994
4,1/8/2016,217.860001,220.440002,210.770004,211.000000,3628100,211.000000


In [ ]:
musk = pd.read_csv('/content/drive/MyDrive/INFOSYS765/Tweets_elon_musk_2016.csv',encoding='latin1')
musk.head()

,ï»¿row ID,Tweet,Time,Retweet from,User
0,Row1083,HW2 Autopilot software uploading to 1000 cars ...,12/31/2016,NaN,elonmusk
1,Row1084,"@vicentes @DragTimes Late Jan, along with Linu...",12/31/2016,NaN,elonmusk
2,Row1085,"@DragTimes Yes, but held up by Autopilot . It ...",12/31/2016,NaN,elonmusk
3,Row1086,Resolving an Autopilot HW2 bug that shows up w...,12/31/2016,NaN,elonmusk
4,Row1087,Churchill (non) quotes \nhttps://t.co/avA4YD6K6g,12/30/2016,NaN,elonmusk


In [ ]:
tweets_by_date = musk.groupby('Time')['Tweet'].apply(' '.join)
tweets_by_date = pd.DataFrame(tweets_by_date)
tweets_by_date.reset_index(inplace = True)

In [ ]:
tweets_by_date.shape

(236, 2)

In [ ]:
tweets_by_date.head()

,Time,Tweet
0,1/1/2016,Falcon 9 back in the hangar at Cape Canaveral....
1,1/10/2016,"In ~2 years, summon should work anywhere conne..."
2,1/11/2016,Sad to hear that David Bowie died. He was amaz...
3,1/12/2016,Next launch is a @NASA science mission from VA...
4,1/16/2016,RT @SpaceX: Rocket is vertical in advance of t...


**Q1 - 1. Sentiment relevant labels using FinBert**

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

In [ ]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

In [ ]:
nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)

In [ ]:
results = nlp(tweets_by_date["Tweet"].to_list())

In [ ]:
tweets_by_date['sentiments'] = results
tweets_by_date['sentiments'] = tweets_by_date['sentiments'].apply(lambda x: x['label'])

In [ ]:
label_map = {'Negative': 0, "Neutral": 1, "Positive": 2}
tweets_by_date["positive"] = tweets_by_date["sentiments"].map(label_map)

In [ ]:
tweets_by_date.head()

,Time,Tweet,sentiments,positive
0,1/1/2016,Falcon 9 back in the hangar at Cape Canaveral....,Neutral,1
1,1/10/2016,"In ~2 years, summon should work anywhere conne...",Neutral,1
2,1/11/2016,Sad to hear that David Bowie died. He was amaz...,Positive,2
3,1/12/2016,Next launch is a @NASA science mission from VA...,Neutral,1
4,1/16/2016,RT @SpaceX: Rocket is vertical in advance of t...,Positive,2


**Q1 -2. ESG featurs from FinBert**

In [ ]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-esg',num_labels=4)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-esg')

In [ ]:
nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)

In [ ]:
results = nlp(tweets_by_date["Tweet"].to_list())

In [ ]:
tweets_by_date['ESG'] = results
tweets_by_date['ESG'] = tweets_by_date['ESG'].apply(lambda x: x['label'])

**Q1 -3. FLG featurs from FinBert**

In [ ]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-fls',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-fls')

In [ ]:
nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)

In [ ]:
results = nlp(tweets_by_date["Tweet"].to_list())

In [ ]:
tweets_by_date['FLG'] = results
tweets_by_date['FLG'] = tweets_by_date['FLG'].apply(lambda x: x['label'])

In [ ]:
df1 = pd.get_dummies(tweets_by_date['ESG'])
df2 = pd.get_dummies(tweets_by_date['FLG'])
tweets_by_date = pd.concat([tweets_by_date, df1], axis = 1)
tweets_by_date = pd.concat([tweets_by_date, df2], axis = 1)

In [ ]:
tweets_by_date.head()

,Time,Tweet,sentiments,positive,ESG,FLG,Environmental,None,Social,Non-specific FLS,Not FLS,Specific FLS
0,1/1/2016,Falcon 9 back in the hangar at Cape Canaveral....,Neutral,1,None,Not FLS,0,1,0,0,1,0
1,1/10/2016,"In ~2 years, summon should work anywhere conne...",Neutral,1,Environmental,Specific FLS,1,0,0,0,0,1
2,1/11/2016,Sad to hear that David Bowie died. He was amaz...,Positive,2,Social,Not FLS,0,0,1,0,1,0
3,1/12/2016,Next launch is a @NASA science mission from VA...,Neutral,1,Social,Specific FLS,0,0,1,0,0,1
4,1/16/2016,RT @SpaceX: Rocket is vertical in advance of t...,Positive,2,None,Not FLS,0,1,0,0,1,0


**Q1 -4. The length of tweets, number of stopwords, and the average word length**

In [ ]:
df = tweets_by_date.copy()
df.head()

,Time,Tweet,sentiments,positive,ESG,FLG,Environmental,None,Social,Non-specific FLS,Not FLS,Specific FLS
0,1/1/2016,Falcon 9 back in the hangar at Cape Canaveral....,Neutral,1,None,Not FLS,0,1,0,0,1,0
1,1/10/2016,"In ~2 years, summon should work anywhere conne...",Neutral,1,Environmental,Specific FLS,1,0,0,0,0,1
2,1/11/2016,Sad to hear that David Bowie died. He was amaz...,Positive,2,Social,Not FLS,0,0,1,0,1,0
3,1/12/2016,Next launch is a @NASA science mission from VA...,Neutral,1,Social,Specific FLS,0,0,1,0,0,1
4,1/16/2016,RT @SpaceX: Rocket is vertical in advance of t...,Positive,2,None,Not FLS,0,1,0,0,1,0


In [ ]:
df = df.dropna()

df.Tweet = df.Tweet.str.lower()

df.Tweet = df.Tweet.apply(lambda x:re.sub('@[^\s]+','',x))

df.Tweet = df.Tweet.apply(lambda x:re.sub(r"http\S+", "", x))

df.Tweet = df.Tweet.apply(lambda x:' '.join(re.findall(r'\w+', x)))

df.Tweet = df.Tweet.apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', '', x))

df.Tweet = df.Tweet.apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))

df.head()

,Time,Tweet,sentiments,positive,ESG,FLG,Environmental,None,Social,Non-specific FLS,Not FLS,Specific FLS
0,1/1/2016,falcon 9 back in the hangar at cape canaveral ...,Neutral,1,None,Not FLS,0,1,0,0,1,0
1,1/10/2016,in 2 years summon should work anywhere connect...,Neutral,1,Environmental,Specific FLS,1,0,0,0,0,1
2,1/11/2016,sad to hear that david bowie died he was amazing,Positive,2,Social,Not FLS,0,0,1,0,1,0
3,1/12/2016,next launch isscience mission from vafb califo...,Neutral,1,Social,Specific FLS,0,0,1,0,0,1
4,1/16/2016,rt rocket is vertical in advance of tomorrow1 ...,Positive,2,None,Not FLS,0,1,0,0,1,0


In [ ]:
df['# Of Words']          = df['Tweet'].apply(lambda x: len(x.split(' ')))
df['# Of StopWords']      = df['Tweet'].apply(lambda x: len([word for word in x.split(' ') if word in list(STOPWORDS)]))
df['Average Word Length'] = df['Tweet'].apply(lambda x: np.mean(np.array([len(va) for va in x.split(' ') if va not in list(STOPWORDS)])))

/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


**Q1 -4. The number of currencies and organizations mentioned in tweets**

In [ ]:
df['# Of Times Currency Was Mentioned']          = df['Tweet'].apply(lambda x: len([tok for tok in nlps(x).ents if tok.label_ == 'MONEY' ]))
df['# Of Organizations Mentioned']           = df['Tweet'].apply(lambda x: len([tok for tok in nlps(x).ents if tok.label_ == 'ORG' ]))

**Q1 -5. The year, month, day, day of year, quarter, and season the tweets were made**

In [ ]:
df['Time'] = pd.to_datetime(df['Time'])
df.head()

,Time,Tweet,sentiments,positive,ESG,FLG,Environmental,None,Social,Non-specific FLS,Not FLS,Specific FLS,# Of Words,# Of StopWords,Average Word Length,# Of Times Currency Was Mentioned,# Of Organizations Mentioned
0,2016-01-01,falcon 9 back in the hangar at cape canaveral ...,Neutral,1,None,Not FLS,0,1,0,0,1,0,16,6,5.000000,0,1
1,2016-01-10,in 2 years summon should work anywhere connect...,Neutral,1,Environmental,Specific FLS,1,0,0,0,0,1,68,28,4.600000,0,0
2,2016-01-11,sad to hear that david bowie died he was amazing,Positive,2,Social,Not FLS,0,0,1,0,1,0,10,4,4.666667,0,0
3,2016-01-12,next launch isscience mission from vafb califo...,Neutral,1,Social,Specific FLS,0,0,1,0,0,1,27,8,6.578947,0,0
4,2016-01-16,rt rocket is vertical in advance of tomorrow1 ...,Positive,2,None,Not FLS,0,1,0,0,1,0,53,12,5.731707,0,1


In [ ]:
#Sorting And Feature Engineering
f_data = df.sort_values(by='Time')
ft_data=f_data.copy()
ft_data = ft_data.rename(columns={'Time':'date'})
ft_data['year']         = pd.DatetimeIndex(ft_data['date']).year
ft_data['month']        = pd.DatetimeIndex(ft_data['date']).month
ft_data['day']          = pd.DatetimeIndex(ft_data['date']).day
ft_data['day_of_year']  = pd.DatetimeIndex(ft_data['date']).dayofyear
ft_data['quarter']      = pd.DatetimeIndex(ft_data['date']).quarter
ft_data['season']       = ft_data.month%12 // 3 + 1

**Other NER-related features**

**Q1 - 8. Calculate the number of tweets per day**

In [ ]:
num_tweets = musk['Time'].value_counts()
num_tweets = pd.DataFrame(num_tweets)
num_tweets.reset_index(inplace = True)
num_tweets.rename(columns = {'index':'Time', 'Time': 'Tweet_num'}, inplace = True)

In [ ]:
tweets_by_date = pd.merge(tweets_by_date, num_tweets, on='Time', how='left')

In [ ]:
tweets_by_date

,Time,Tweet,sentiments,positive,ESG,FLG,Environmental,None,Social,Non-specific FLS,Not FLS,Specific FLS,Tweet_num
0,1/1/2016,Falcon 9 back in the hangar at Cape Canaveral....,Neutral,1,None,Not FLS,0,1,0,0,1,0,1
1,1/10/2016,"In ~2 years, summon should work anywhere conne...",Neutral,1,Environmental,Specific FLS,1,0,0,0,0,1,3
2,1/11/2016,Sad to hear that David Bowie died. He was amaz...,Positive,2,Social,Not FLS,0,0,1,0,1,0,1
3,1/12/2016,Next launch is a @NASA science mission from VA...,Neutral,1,Social,Specific FLS,0,0,1,0,0,1,2
4,1/16/2016,RT @SpaceX: Rocket is vertical in advance of t...,Positive,2,None,Not FLS,0,1,0,0,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,9/26/2016,@ID_AA_Carmack yup RT @SpaceX: Tomorrow: Elon ...,Neutral,1,Environmental,Specific FLS,1,0,0,0,0,1,10
232,9/27/2016,Waiting for everyone to reach their seats and ...,Neutral,1,None,Specific FLS,0,1,0,0,0,1,5
233,9/28/2016,@GoodDayM Am tracking this down ... Super impo...,Neutral,1,Social,Not FLS,0,0,1,0,1,0,9
234,9/29/2016,RT @waitbutwhy: The full story with the mindbl...,Neutral,1,Social,Not FLS,0,0,1,0,1,0,2


In [ ]:
tesla_tw['Date'] = pd.to_datetime(tesla_tw['Date'])

**Q2 - Merge stock price data with tweet data**

In [ ]:
data = pd.merge(tesla_tw, ft_data, left_on='Date', right_on='date', how='inner')
data.head()

,Date,Open,High,Low,Close,Volume,Adj Close,date,Tweet,sentiments,...,# Of StopWords,Average Word Length,# Of Times Currency Was Mentioned,# Of Organizations Mentioned,year,month,day,day_of_year,quarter,season
0,2016-01-11,214.009995,214.449997,203.000000,207.850006,4091400,207.850006,2016-01-11,sad to hear that david bowie died he was amazing,Positive,...,4,4.666667,0,0,2016,1,11,11,1,1
1,2016-01-12,211.600006,213.740005,205.309998,209.970001,3091900,209.970001,2016-01-12,next launch isscience mission from vafb califo...,Neutral,...,8,6.578947,0,0,2016,1,12,12,1,1
2,2016-01-19,208.710007,210.470001,200.779999,204.720001,4038700,204.720001,2016-01-19,my best guess for 2016 70 landing success rate...,Positive,...,8,5.071429,0,0,2016,1,19,19,1,1
3,2016-01-20,199.399994,201.279999,191.250000,198.699997,5838600,198.699997,2016-01-20,i didntake anyonecar and never would car was a...,Neutral,...,9,7.400000,0,0,2016,1,20,20,1,1
4,2016-01-21,201.550003,203.229996,195.020004,199.970001,3166200,199.970001,2016-01-21,rt 2015 was the warmest year on record have qs...,Neutral,...,5,4.454545,0,0,2016,1,21,21,1,1


In [ ]:
data.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close', 'date',
       'Tweet', 'sentiments', 'positive', 'ESG', 'FLG', 'Environmental',
       'None', 'Social', 'Non-specific FLS', 'Not FLS', 'Specific FLS',
       '# Of Words', '# Of StopWords', 'Average Word Length',
       '# Of Times Currency Was Mentioned', '# Of Organizations Mentioned',
       'year', 'month', 'day', 'day_of_year', 'quarter', 'season'],
      dtype='object')

**Predict closing price using linear regression**

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Define the features and target variable
X = data[['positive','Environmental','Social','Non-specific FLS',
          'Specific FLS','# Of Words', '# Of StopWords', '# Of Organizations Mentioned']]
y = data['Close']

# Create a Linear Regression model and fit it to the training data
lr_model = LinearRegression()

In the prediction, I picked eight columns as the inputs. The reason why I did not pick `None` and `Not FLS` and other timestamp variables are to avoid multi-collinearity.

**Cross-validation**

In [ ]:
# Evaluate the model's performance on the test data
scores = cross_val_score(lr_model, X, y, cv=5)

print("R-Squared of the model is", scores.mean(), "with a standard deviation of", scores.std())

R-Squared of the model is -1.769531268280878 with a standard deviation of 1.0898824722904226


However, the R-squared is only -1.769531. We then inspect the significance of each column we have chosen.

**Q3 - Tune the prediction model and try to improve the model performance**

In [ ]:
import statsmodels.api as sm

# Add a constant term to the X data to represent the intercept
X = sm.add_constant(X)

# Create a Linear Regression model
lr_model = sm.OLS(y, X)

# Fit the model to the data
results = lr_model.fit()

# Print the summary of the model
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  Close   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.9744
Date:                Tue, 21 Mar 2023   Prob (F-statistic):              0.458
Time:                        22:49:15   Log-Likelihood:                -740.98
No. Observations:                 164   AIC:                             1500.
Df Residuals:                     155   BIC:                             1528.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const           

The significance result suggests that except the `positive` column, other attributes are all not significance under a 95% confidence level. Therefore, we only keep the `positive` column and tune others to see if the performance could be improved.

In [ ]:
# Define the features and target variable
X = data[['positive']]
y = data['Close']
lr_model = LinearRegression()
scores = cross_val_score(lr_model, X, y, cv=5)

print("R-Squared of the model is", scores.mean(), "with a standard deviation of", scores.std())

R-Squared of the model is -1.3515067575018374 with a standard deviation of 1.0410810928506045


In [ ]:
# Add a constant term to the X data to represent the intercept
X = sm.add_constant(X)

# Create a Linear Regression model
lr_model = sm.OLS(y, X)

# Fit the model to the data
results = lr_model.fit()

# Print the summary of the model
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  Close   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     3.429
Date:                Tue, 21 Mar 2023   Prob (F-statistic):             0.0659
Time:                        22:50:41   Log-Likelihood:                -743.29
No. Observations:                 164   AIC:                             1491.
Df Residuals:                     162   BIC:                             1497.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        219.4365      5.277     41.582      0.0

The R-squared after tuning has improved to -1.3515. However, the `positive` column is now insignificant under a 95% significance level, which does not suggest a strong linear relationship between stock's close prices and sentiment scores. To better predict stock price, I am planning to use lagged stock prices as the dependent variables. The reason behind is that there is a time lag effects on the stock prices after tweets have been released. Therefore, the instant effects on stocks prices might not be captured exactly on the specific day. As for the features to be used as independent variables, I plan to use the trading volume, standard deviation of stock price, and risk-free market return to predict tesla's stock price. Because these factors have the direct influence on the stock's prices and can be indirectly affected by Elon Musk's tweets.